<a href="https://colab.research.google.com/github/MayerT1/ACI_Collaboration/blob/main/ACI_cerulean_warbler_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pygbif

In [ ]:
import ee
import geemap
import pandas as pd
from pygbif import occurrences
import geemap.chart as chart

In [ ]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='servir-sco-assets')
Map = geemap.Map()

https://www.gbif.org/species/6093350

In [ ]:
# Define AOI: Tennessee
states = ee.FeatureCollection("TIGER/2018/States")
tennessee = states.filter(ee.Filter.eq('NAME', 'Tennessee'))
roi = tennessee.geometry()
Map = geemap.Map(center=[35, -86], zoom=6)
Map.addLayer(tennessee, {}, "Tennessee AOI")

# --------------------------------------------------
# Fetch Cerulean Warbler occurrences from GBIF
# --------------------------------------------------
species_name = "Setophaga cerulea"
gbif_data = occurrences.search(scientificName=species_name, country="US", stateProvince="Tennessee", hasCoordinate=True, limit=3000)

# Convert GBIF occurrences to a DataFrame
records = gbif_data['results']
df = pd.DataFrame(records)
df = df.dropna(subset=['decimalLatitude', 'decimalLongitude'])

# Convert to ee.FeatureCollection
def df_to_fc(row):
    return ee.Feature(ee.Geometry.Point([row['decimalLongitude'], row['decimalLatitude']]), {'occurrence': 1})

presence_fc = ee.FeatureCollection(df.apply(df_to_fc, axis=1).tolist())
Map.addLayer(presence_fc, {'color': 'blue'}, "Presence Points")

# --------------------------------------------------
# Generate background (pseudo-absence) points
# --------------------------------------------------
background_fc = ee.FeatureCollection.randomPoints(roi, 10000, seed=42)
background_fc = background_fc.map(lambda f: f.set('occurrence', 0))
Map.addLayer(background_fc, {'color': 'red'}, "Background Points")

# Merge presence and background points
training_data = presence_fc.merge(background_fc)
Map.addLayer(training_data, {}, "Training Data")

# --------------------------------------------------
# Landsat 8 imagery preprocessing
# --------------------------------------------------
def maskL8sr(image):
    qa = image.select('QA_PIXEL')
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
        qa.bitwiseAnd(cloudsBitMask).eq(0)
    )
    return image.updateMask(mask)

LS8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterDate('2020-01-01', '2020-12-31') \
        .filterBounds(roi) \
        .map(maskL8sr)

# Compute NDVI median image
def add_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return ndvi.copyProperties(image).set('system:time_start', image.get('system:time_start'))

image = LS8.map(add_ndvi).median().clip(roi)
Map.addLayer(image, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, "NDVI")

# --------------------------------------------------
# Sample points and train Maxent classifier
# --------------------------------------------------
training = image.sampleRegions(
    collection=training_data,
    scale=30
)

classifier = ee.Classifier.amnhMaxent().train(
    features=training,
    classProperty='occurrence',
    inputProperties=image.bandNames()
)

# Classify image
image_classified = image.classify(classifier)
Map.addLayer(image_classified.select('probability'), {'min':0, 'max':1, 'palette': ['blue','white','red']}, 'Predicted Probability')

# Display map
Map
